In [18]:
import pandas as pd
import requests
import json,math
from datetime import datetime as dt
import threading
import time
from datetime import timedelta
import numpy as np
import re

# Function to extract data from url
def data_from_url (urls):
  #start = dt.datetime.now()
  df = pd.DataFrame() 
  # url1 to store urls facing error_code 429 : Api limit exceeds  
  url1 = list()
  
  for url in urls:
    #time.sleep(0.)    
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
            data = pd.DataFrame(json.loads(response.text), columns=['time', 'low', 'high', 'open', 'close', 'volume'])
            time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(str(epoch_time)[0:10])))
            #data['date'] = pd.to_datetime(time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(str(data['time'])[0:10]))))
            #data['date'] = pd.to_datetime(str(data['time'])[0:10], unit='s')  # convert to a readable date
            #data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume
            #if len(data.index) == 0:
             #  print('XOXO')
              # Can print urls where for which coinbase doesn't have data
              # print(str(len(data.index))+ " " + url + '\n')
            df = df.append(data)
    # when error code = 429, Again call data_from_url
    elif response.status_code == 429:
         url1.append(url)
         response1 = requests.get(url)
         data = pd.DataFrame(json.loads(response1.text), columns=['time', 'low', 'high', 'open', 'close', 'volume'])
         df = df.append(data)
         print(f'{response.status_code} ^^^ Error in 429')
    elif response.status_code == 504:
        #url1.append(url)
        print(f'{response.status_code} -- Error in 504 ' + url + '\n')
        time.sleep(0.1)    
        response1 = requests.get(url)
        data = pd.DataFrame(json.loads(response1.text), columns=['time', 'low', 'high', 'open', 'close', 'volume'])
        df = df.append(data)
        ##print(str(len(data.index))+ " " + url + '\n')
    else:
        print("Did not receieve OK response from Coinbase API:" + url)
        print(f'{response.status_code}: Error in else')
    end = dt.now()
    
  if len(url1) > 0:
    data_from_url(url1)
  #end = dt.datetime.now()
  print('END of urls')
  return df      

# Function generated all urls for given currency
url = "https://public.coindcx.com/market_data/candles?pair=B-BTC_USDT&interval=1m" 
def generate_urls_from_symbol_name(range_end,symbol,epoch_startDate,epoch_endDate):
    # List to store all url
    urls = []
    interval = '5m'
    for i in range(0,range_end):
        url_endDate = int((dt.fromtimestamp(epoch_startDate) + timedelta(minutes =5000)).timestamp())
        
        if epoch_endDate > url_endDate:
            urls.append(f"https://public.coindcx.com/market_data/candles?pair={symbol}&interval={interval}&startTime={int(epoch_startDate)}000&endTime={int(url_endDate)}000&limit=1000")

            #urls.append(f'https://public.coindcx.com/market_data/candles?pair={symbol}&starttime={int(epoch_startDate)}000&endtime={url_endDate}000&interval=5m&limit=1000')
            epoch_startDate = url_endDate
        elif epoch_endDate < url_endDate:
            #epoch_endDate = dt.utcfromtimestamp(epoch_endDate).strftime('%Y-%m-%dT%H:%M:%SZ')
            urls.append(f"https://public.coindcx.com/market_data/candles?pair={symbol}&interval={interval}&startTime={int(epoch_startDate)}000&endTime={int(epoch_endDate)}000&limit=1000")

    return urls

def fetch_daily_data(pair,range_end,epoch_startDate,epoch_endDate):
    # Handle Day-light saving changes
  #if epoch_endDate == '2021-11-07T16:00:00':
   # epoch_endDate = '2021-11-07T15:00:00'
  for p in pair:  
    urls = generate_urls_from_symbol_name(range_end,p,epoch_startDate,epoch_endDate)
    data = data_from_url(urls)
    # if we failed to get any data, print an error...otherwise write the file
    if data is None:
            print("Did not return any data from Coinbase for this symbol:"+ p  )
    else:
            data.to_csv(f'CoinDCX_{p}_dailydata.csv', index=False)

if __name__ == "__main__":
    #url = "https://api.coindcx.com/exchange/v1/markets_details"
    #response = requests.get(url).json()
    #this_list = []
    #for i in range(len(response)):
     #   if re.search('^B-.*USDT$',response[i]['pair']):
      #      this_list.append(response[i]['pair'])
    # we set which pair we want to retrieve data for
    #pair = ['BTC-USD', 'BTC-USDT', 'DOGE-USD', 'ETH-USD', 'ETH-EUR']
    #pair = ['BTC-USD', 'BTC-USDT', 'DOGE-USD', 'ETH-USD', 'ETH-EUR','BTC-USD', 'ETH-GBP', 'BTC-GBP', 'LTC-EUR', 'LTC-USD', 'LTC-GBP']
    #pair = ["ZRX-USD","1INCH-USD","AAVE-USD","ALCX-USD","ACH-USD","AGLD-USD"]
    pair1 = ['B-BTC_USDT','B-ACA_USDT', 'B-DENT_USDT', 'B-REEF_USDT', 'B-CLV_USDT', 'B-SPELL_USDT', 'B-CHESS_USDT',
     'B-MITH_USDT', 'B-FIRO_USDT','B-BNX_USDT', 'B-MDX_USDT']
    pair = ['B-BTC_USDT','B-ETH_USDT', 'B-DOGE_USDT', 'B-1INCH_USDT','B-AAVE_USDT', 'B-ACH_USDT','B-ADA_USDT', 'B-JOE_USDT' ,
            'B-ARPA_USDT', 'B-DOCK_USDT', 'B-BEAM_USDT', 'B-WTC_USDT', 'B-MKR_USDT', 'B-BETA_USDT','B-YGG_USDT', 'B-CHZ_USDT', 
            'B-REQ_USDT', 'B-CTXC_USDT', 'B-POLY_USDT', 'B-LTC_USDT']
    epoch_startDate = dt(2021,6,1).timestamp()
    epoch_endDate = dt(2022,3,10).timestamp()
    
    range_end = math.ceil((epoch_endDate-epoch_startDate)/(60000*5))
    print(range_end, len(pair))
    # Declare number of threads
    parallel_threads = 10
    #Splitting Pair files for each thread
    parallel_arrays = np.array_split(pair, parallel_threads)
    workers = list()
  
    start = dt.now()
    #Process Pair Files parallely
    for k in range(parallel_threads):
        # Implemented MultiThreading to process 3 threads in parallel
        wp = threading.Thread(target=fetch_daily_data, name='worker%d' % (k + 1), args=(list(parallel_arrays[k]),range_end,epoch_startDate,epoch_endDate))
        workers.append(wp)
        wp.start()
   
    #Waitng for threads to complete
    for wp in workers:
        wp.join()
    end = dt.now()
    # Print total Time to extract Data
    print('END of program')
    print(start, end , end-start)

82 20
504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-ARPA_USDT&interval=5m&startTime=1627023600000&endTime=1627323600000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-ADA_USDT&interval=5m&startTime=1627323600000&endTime=1627623600000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-REQ_USDT&interval=5m&startTime=1633323600000&endTime=1633623600000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-YGG_USDT&interval=5m&startTime=1638727200000&endTime=1639027200000&limit=1000

END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-WTC_USDT&interval=5m&startTime=1635423600000&endTime=1635723600000&limit=1000

END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
END of urls


In [1]:
import pandas as pd
import requests
import json,math
from datetime import datetime as dt
import threading
import time
from datetime import timedelta
import numpy as np
import re

# Function to extract data from url
def data_from_url (urls):
  #start = dt.datetime.now()
  df = pd.DataFrame() 
  # url1 to store urls facing error_code 429 : Api limit exceeds  
  url1 = list()
  
  for url in urls:
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
            data = pd.DataFrame(json.loads(response.text), columns=['time', 'low', 'high', 'open', 'close', 'volume'])
            #data['date'] = pd.to_datetime(time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(str(data['time'])[0:10]))))
            #data['date'] = pd.to_datetime(str(data['time'])[0:10], unit='s')  # convert to a readable date
            #if len(data.index) == 0:
             #  print('XOXO')
              # Can print urls where for which coinbase doesn't have data
              # print(str(len(data.index))+ " " + url + '\n')
            df = df.append(data)
    # when error code = 429, Again call data_from_url
    elif response.status_code == 504:
        print(f'{response.status_code} -- Error in 504 ' + url + '\n')
        time.sleep(0.1)    
        response1 = requests.get(url)
        if response1.status_code == 200:
            data = pd.DataFrame(json.loads(response1.text), columns=['time', 'low', 'high', 'open', 'close', 'volume'])
            df = df.append(data)
        else:
            print("Did not receieve OK response from Coinbase API:" + url)
            print(f'{response.status_code}: Error in else of 504')
    else:
        print("Did not receieve OK response from Coinbase API:" + url)
        print(f'{response.status_code}: Error in else')
    end = dt.now()
    
  if len(url1) > 0:
    data_from_url(url1)
  #end = dt.datetime.now()
  print('END of urls')
  return df      

# Function generated all urls for given currency
url = "https://public.coindcx.com/market_data/candles?pair=B-BTC_USDT&interval=1m" 
def generate_urls_from_symbol_name(range_end,symbol,epoch_startDate,epoch_endDate):
    # List to store all url
    urls = []
    interval = '5m'
    for i in range(0,range_end):
        url_endDate = int((dt.fromtimestamp(epoch_startDate) + timedelta(minutes =5000)).timestamp())
        
        if epoch_endDate > url_endDate:
            urls.append(f"https://public.coindcx.com/market_data/candles?pair={symbol}&interval={interval}&startTime={int(epoch_startDate)}000&endTime={int(url_endDate)}000&limit=1000")

            #urls.append(f'https://public.coindcx.com/market_data/candles?pair={symbol}&starttime={int(epoch_startDate)}000&endtime={url_endDate}000&interval=5m&limit=1000')
            epoch_startDate = url_endDate
        elif epoch_endDate < url_endDate:
            #epoch_endDate = dt.utcfromtimestamp(epoch_endDate).strftime('%Y-%m-%dT%H:%M:%SZ')
            urls.append(f"https://public.coindcx.com/market_data/candles?pair={symbol}&interval={interval}&startTime={int(epoch_startDate)}000&endTime={int(epoch_endDate)}000&limit=1000")

    return urls

def fetch_daily_data(pair,range_end,epoch_startDate,epoch_endDate):
    # Handle Day-light saving changes
  #if epoch_endDate == '2021-11-07T16:00:00':
   # epoch_endDate = '2021-11-07T15:00:00'
  for p in pair:  
    urls = generate_urls_from_symbol_name(range_end,p,epoch_startDate,epoch_endDate)
    data = data_from_url(urls)
    # if we failed to get any data, print an error...otherwise write the file
    if data is None:
            print("Did not return any data from Coinbase for this symbol:"+ p  )
    else:
            data.to_csv(f'CoinDCX_{p}_dailydata.csv', index=False)

if __name__ == "__main__":
    #url = "https://api.coindcx.com/exchange/v1/markets_details"
    #response = requests.get(url).json()
    #this_list = []
    #for i in range(len(response)):
     #   if re.search('^B-.*USDT$',response[i]['pair']):
      #      this_list.append(response[i]['pair'])
    # we set which pair we want to retrieve data for
    #pair = ['BTC-USD', 'BTC-USDT', 'DOGE-USD', 'ETH-USD', 'ETH-EUR']
    #pair = ['BTC-USD', 'BTC-USDT', 'DOGE-USD', 'ETH-USD', 'ETH-EUR','BTC-USD', 'ETH-GBP', 'BTC-GBP', 'LTC-EUR', 'LTC-USD', 'LTC-GBP']
    #pair = ["ZRX-USD","1INCH-USD","AAVE-USD","ALCX-USD","ACH-USD","AGLD-USD"]
    pair1 = ['B-BTC_USDT','B-ACA_USDT', 'B-DENT_USDT', 'B-REEF_USDT', 'B-CLV_USDT', 'B-SPELL_USDT', 'B-CHESS_USDT',
     'B-MITH_USDT', 'B-FIRO_USDT','B-BNX_USDT', 'B-MDX_USDT']
    pair = ['B-ACA_USDT', 'B-DENT_USDT', 'B-REEF_USDT', 'B-CLV_USDT', 'B-SPELL_USDT', 'B-CHESS_USDT', 'B-MITH_USDT', 'B-FIRO_USDT','B-BNX_USDT', 'B-MDX_USDT', 'B-NBS_USDT', 'B-SLP_USDT', 'B-CFX_USDT', 'B-AUTO_USDT', 'B-OOKI_USDT', 'B-COMP_USDT', 'B-CITY_USDT', 'B-LIT_USDT', 'B-TRB_USDT', 'B-BAND_USDT', 'B-UNI_USDT', 'B-RAD_USDT', 'B-BAR_USDT', 'B-RAY_USDT', 'B-XTZ_USDT', 'B-TLM_USDT', 'B-UST_USDT', 'B-CELR_USDT', 'B-POLS_USDT', 'B-BTC_USDT', 'B-FIS_USDT', 'B-BAT_USDT', 'B-KNC_USDT', 'B-ENJ_USDT', 'B-DATA_USDT' , 'B-IOTA_USDT', 'B-GTC_USDT', 'B-IMX_USDT', 'B-KSM_USDT', 'B-DAR_USDT', 'B-HNT_USDT', 'B-ANC_USDT', 'B-IRIS_USDT', 'B-JOE_USDT' , 'B-ARPA_USDT', 'B-DOCK_USDT', 'B-BEAM_USDT', 'B-WTC_USDT', 'B-MKR_USDT', 'B-BETA_USDT', 'B-YGG_USDT', 'B-CHZ_USDT', 'B-REQ_USDT', 'B-CTXC_USDT', 'B-POLY_USDT', 'B-LTC_USDT', 'B-SUN_USDT', 'B-PNT_USDT', 'B-FIL_USDT', 'B-DASH_USDT', 'B-ARDR_USDT', 'B-GHST_USDT', 
            'B-ASTR_USDT', 'B-BTS_USDT', 'B-PERL_USDT', 'B-PSG_USDT', 'B-XVS_USDT', 'B-MFT_USDT', 'B-PUNDIX_USDT', 'B-COS_USDT', 'B-PLA_USDT', 'B-FLM_USDT', 'B-FIO_USDT', 'B-LPT_USDT', 'B-COTI_USDT','B-ONG_USDT', 'B-WOO_USDT', 'B-STX_USDT', 'B-MOVR_USDT', 'B-VET_USDT']
    epoch_startDate = dt(2021,6,1).timestamp()
    epoch_endDate = dt(2022,3,10).timestamp()
    
    range_end = math.ceil((epoch_endDate-epoch_startDate)/(60000*5))
    print(range_end, len(pair))
    # Declare number of threads
    parallel_threads = 10
    #Splitting Pair files for each thread
    parallel_arrays = np.array_split(pair, parallel_threads)
    workers = list()
  
    start = dt.now()
    #Process Pair Files parallely
    for k in range(parallel_threads):
        # Implemented MultiThreading to process 3 threads in parallel
        wp = threading.Thread(target=fetch_daily_data, name='worker%d' % (k + 1), args=(list(parallel_arrays[k]),range_end,epoch_startDate,epoch_endDate))
        workers.append(wp)
        wp.start()
   
    #Waitng for threads to complete
    for wp in workers:
        wp.join()
    end = dt.now()
    # Print total Time to extract Data
    print('END of program')
    print(start, end , end-start)

82 80
504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-BNX_USDT&interval=5m&startTime=1623723600000&endTime=1624023600000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-PERL_USDT&interval=5m&startTime=1624323600000&endTime=1624623600000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-MKR_USDT&interval=5m&startTime=1627623600000&endTime=1627923600000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-KNC_USDT&interval=5m&startTime=1635423600000&endTime=1635723600000&limit=1000

END of urls
END of urls
END of urls
END of urls
END of urls
END of urls
504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-MKR_USDT&interval=5m&startTime=1642027200000&endTime=1642327200000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-PERL_USDT&interval=5m&startTime=1633323600000&endTime=1633623600000&limit=1000

EN